# **Get the mean/average color of each frame and show it as a barcode**

## **ANLEITUNG**
### **[1. Voreinstellung]** Unter "Paths" (Zelle 4) beide Pfade anpassen.
### **[2. Voreinstellung]** In der letzten Zelle (Generate movie barcode) unter "walkthrough_name" den Namen des Walkthroughs anpassen.
### **[Mean/Average-Color aus jedem Frame bekommen]** Die Zelle bei "Get frame list" ausführen. Der aktuelle Stand wird fortlaufend angezeigt.
### **[Bei Abbruch, Verbindungsproblemen, usw.]** Die Zelle bei "Continue process [...]" ausführen. Der aktuelle Stand wird fortlaufend angezeigt.
### **[Wenn Prozess abgeschlossen]** Die Zelle bei "Generate the movie barcode" ausführen. Der MovieBarcode wird in verschiedenen Formaten generiert und abgespeichert.

## **Packages needed to be installed**

In [ ]:
!pip install colorthief #installed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **Imports**

In [ ]:
#from colorthief import ColorThief
from natsort import natsorted, ns
from google.colab.patches import cv2_imshow
# use next line if you want to clear output (right now not needed)
#from IPython.display import clear_output 

import matplotlib.pyplot as plt
import statistics
from statistics import median 
import os
import skimage.io as io
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd

## **Connection to Google Drive**
### Doesn't have to be executed. Alternatively, the mounting can also be initialized manually in the left area.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Paths**
### Enter the path for the frames folder and the folder where the results should be saved.

In [ ]:
# folder in which the frames of the walkthrough are saved (Don't forget the "/" at the end)
path_to_game_frames = "/content/drive/MyDrive/digital_humanities/Walkthroughts/Siedler_Frames/"
# folder in which the results are to be saved (Don't forget the "/" at the end)
path_to_result_data = "/content/drive/MyDrive/digital_humanities/Ergebnisse/Color_Analysis/"

## **Get frame list**
### for each frame the mean/average color is generated and saved in an array 
### save the names of the frames that have already been analyzed (in case the process is aborted)
### show the current status of the process

In [ ]:
# get mean/average color for each frame and save it into the average_color_arr array

# list of frames in the folder
frames_list = os.listdir(path_to_game_frames)
# sorted list of frames in the folder
frames_list = natsorted(frames_list, alg=ns.PATH | ns.IGNORECASE)

# names of already analyzed frames are saved here. Serves to find out at which point the process should continue to run in the event of a process crash.
backup_check = []

brightness_dict = {}
brightness_values_dict = {}
statistic_result_dict = {}

sum_of_brigthness_values = 0

for frame in frames_list:
  if frame.split(".")[-1].lower() in {"jpeg", "jpg"}:
    source_image = cv2.imread(path_to_game_frames + frame)
    greyscale_img = cv2.cvtColor(source_image, cv2.COLOR_BGR2GRAY)
    img_brightness = greyscale_img.mean()/255
    #sum_of_brigthness_values += img_brightness

    brightness_values_dict[frame] = img_brightness

    brightness_dict[frame] = []
    brightness_dict[frame].append(round(img_brightness, 2))
    brightness_dict[frame].append(frame)

    print(f'\rProcessing: {str(len(brightness_values_dict))} of {str(len(frames_list))} Frames have been analyzed.', end='')






## **Continue process after disconnects, time ups, ...**

In [ ]:
mean_brightness = sum(brightness_values_dict.values()) / len(brightness_dict)
sd_brightness = statistics.stdev(brightness_values_dict.values())
median_brightness = statistics.stdev(brightness_values_dict.values())
min_brightness = min(brightness_values_dict.values())
max_brightness = max(brightness_values_dict.values())

statistic_result_dict = {}
statistic_result_dict["min"] = []
statistic_result_dict["min"].append(round(min_brightness, 2))
statistic_result_dict["median"] = []
statistic_result_dict["median"].append(round(median_brightness, 2))
statistic_result_dict["mean"] = []
statistic_result_dict["mean"].append(round(mean_brightness, 2))
statistic_result_dict["sd"] = []
statistic_result_dict["sd"].append(round(sd_brightness, 2))
statistic_result_dict["max"] = []
statistic_result_dict["max"].append(round(max_brightness, 2))


dataBrightness = pd.DataFrame.from_dict(brightness_dict, orient='index', columns=["brightness_value", "frame"])
#dataResults = pd.DataFrame.from_dict(statistic_result_dict, orient='index', columns=["max", "min", "median", "mean", "sd"])
dataResults = pd.DataFrame.from_dict(statistic_result_dict)

cols = dataBrightness.columns.tolist()
cols = cols[-1:] + cols[:-1]
dataBrightness = dataBrightness[cols]
# Sichern der Daten
path = path_to_result_data + "brightness.csv"
#setattr(dataBrightness ,"dataVis", dataBrightness)
dataBrightness.to_csv(path, encoding="utf-8", index=False)

cols = dataResults.columns.tolist()
cols = cols[-1:] + cols[:-1]
dataResults = dataResults[cols]
# Sichern der Daten
path = path_to_result_data + "results.csv"
#setattr(dataBrightness ,"dataVis", dataBrightness)
dataResults.to_csv(path, encoding="utf-8", index=False)

## **Generate the movie barcode**
### and saves as vector graphics (.eps, .svg) and .png and .jpg